In [1]:
from utils import first_block_of_day, last_block_of_day, rpc_to_parity
from utils import drop_table, create_action_table
from utils import exeSQL, exeMultipleSQL, insert_action, insert_multiple_actions
from tqdm import tqdm_notebook as tqdm

date_start = '2016-10-01'
date_end = '2016-12-31'
block_start = first_block_of_day(int(date_start[:4]), int(date_start[5:7]), int(date_start[8:10]))
block_end = last_block_of_day(int(date_end[:4]), int(date_end[5:7]), int(date_end[8:10]))

table_name = "action_20161001_20161231"

In [2]:
print (block_start)
print (block_end)

2357373
2912406


In [11]:
recreate = True
if recreate:
    drop_table('action_20161001_20161231')
    create_action_table('action_20161001_20161231')

In [4]:
def parse_action_create(dict_obj):
    if 'error' in dict_obj:
        return None
    directive = dict_obj['type']
    tx = dict_obj['transactionHash']
    block_num = dict_obj['blockNumber']
    tx_seq = dict_obj['transactionPosition']
    act_seq = 0
    dict_action = dict_obj['action']
    dict_result = dict_obj['result']
    source = dict_action['from']
    target = dict_result['address']
    amount = dict_action['value']
    parsed_entry = [directive, source, target, amount, tx, block_num, tx_seq, act_seq]
    return parsed_entry

In [5]:
def parse_action_call(dict_obj):
    if 'error' in dict_obj:
        return None
    directive = dict_obj['type']
    tx = dict_obj['transactionHash']
    block_num = dict_obj['blockNumber']
    tx_seq = dict_obj['transactionPosition']
    act_seq = 0
    dict_action = dict_obj['action']
    source = dict_action['from']
    target = dict_action['to']
    amount = dict_action['value']
    parsed_entry = [directive, source, target, amount, tx, block_num, tx_seq, act_seq]
    return parsed_entry

In [6]:
def parse_action_reward(dict_obj):
    dict_action = dict_obj['action']
    directive = dict_obj['type'] + "-" + dict_action['rewardType']
    tx = dict_obj['transactionHash']
    block_num = dict_obj['blockNumber']
    tx_seq = -1
    act_seq = 0
    source = 'None'
    target = dict_action['author']
    amount = dict_action['value']
    parsed_entry = [directive, source, target, amount, tx, block_num, tx_seq, act_seq]
    return parsed_entry

In [7]:
def fetch_entries_from_actions(actions):
    parsed_entries = []
    for act in actions:
        if act['type'] == 'call':
            parsed_entry = parse_action_call(act)
        elif act['type'] == 'create':
            parsed_entry = parse_action_create(act)
        elif act['type'] == 'reward':
            parsed_entry = parse_action_reward(act)
        else:
            print(act)
        if parsed_entry != None:
            parsed_entries.append(parsed_entry)
    last_tx_hash = ''
    act_seq = -1
    for en in parsed_entries:
        if en[4] != last_tx_hash:
            last_tx_hash=  en[4]
            act_seq = 0
        else:
            act_seq += 1
        en[7] = act_seq
    return parsed_entries

In [8]:
def delete_error_actions(actions):
    err_txs_list = []
    for act in actions:
        if 'error' in act:
            err_txs_list.append(act['transactionHash'])
    tailored_actions_list = []
    for act in actions:
        if not act['transactionHash'] in err_txs_list:
            tailored_actions_list.append(act)
    return tailored_actions_list

In [9]:
def parse_blocks(block_start, block_end, table_name):
    for bn in tqdm(range(block_start, block_end+1)):
#         print ("######### Start to process block {} #########".format(bn))
        method = 'trace_block'
        params = [hex(bn)]
        actions_per_block = rpc_to_parity(method, params).json()['result']
#         print (actions_per_block)
        tailored_actions = delete_error_actions(actions_per_block)
        parsed_entries = fetch_entries_from_actions(tailored_actions)
        insert_multiple_actions(parsed_entries, table_name = table_name)

In [ ]:
parse_blocks(block_start, block_end, table_name)

In [142]:
method = 'trace_block'
params = [hex(2357441)]
resp = rpc_to_parity(method, params)
print (resp.json()['result'])

[{'blockHash': '0x10c10eda256b5956af5651d7758575396cedbc97de6250221854ad6e8ff4e18a', 'traceAddress': [], 'blockNumber': 2357441, 'transactionPosition': 0, 'subtraces': 0, 'type': 'create', 'action': {'init': '0x6060604052600261010860005055604051611b51380380611b51833981016040528080518201919060200180519060200190919080519060200190919050505b805b83835b600060018351016001600050819055503373ffffffffffffffffffffffffffffffffffffffff16600260005060016101008110156100025790900160005b5081905550600161010260005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005081905550600090505b825181101561016e5782818151811015610002579060200190602002015173ffffffffffffffffffffffffffffffffffffffff166002600050826002016101008110156100025790900160005b508190555080600201610102600050600085848151811015610002579060200190602002015173ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600050819055505b80600101905080506100c2565b816000600050819055505b505050806101056000508190555061018

In [ ]:
method = 'trace_transaction'
params = ["0x772bb0cc2540c997f100f1c11dbafc7c60808775eeb6f939e096738ce29644b7"]
resp_trace_tx = rpc_to_parity(method, params)
print (resp_trace_tx.json()['result'])

In [167]:
method = 'eth_getBlockByNumber'
params = [hex(2357374), True]
resp = rpc_to_parity(method, params)
print (resp.json()['result'])

{'hash': '0x2ab8efe7da15fce005a385e39ab3a7ee0dc2b8e639de66b58f0db599212d66d3', 'number': '0x23f87e', 'parentHash': '0xd074e73c4dca9e1769c5f1bc016b23a9c0f5d96ab44a453d369319c89d9b6064', 'author': '0xf3b9d2c81f2b24b0fa0acaaa865b7d9ced5fc2fb', 'gasUsed': '0x14820', 'timestamp': '0x57eefc85', 'size': '0x3de', 'stateRoot': '0x4fc1793f8af71dfaec37fffffcd05c1b75df585fdd4a7b7eb81ccf7c6bd2e374', 'totalDifficulty': '0x3ae899048133fd3e1', 'mixHash': '0xa690321199f9175d001863ad0e78fddb03cfc9e5202762957402044c48244acd', 'gasLimit': '0x16e358', 'logsBloom': '0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [13]:
method = 'trace_replayTransaction'
params = ["0x77656b0b74ea1f8b158067c8371617aae7ab06f05c6c13001606a763688602c3", ['trace']]
resp_trace_replay_tx = rpc_to_parity(method, params)
print (resp_trace_replay_tx.json()['result'])

{'output': '0x0000000000000000000000000000000000000000000000000000000000000001', 'stateDiff': None, 'trace': [{'subtraces': 0, 'type': 'call', 'action': {'input': '0xa9059cbb0000000000000000000000000328307a6cab168e3947bfaec99bf5b64573383f00000000000000000000000000000000000000000000000c9a80665ea1ae5000', 'gas': '0x1eed8', 'from': '0xd9bd20efca7b0e6606b969548b1516c08d37374b', 'value': '0x0', 'to': '0xaf30d2a7e90d7dc361c8c4585e9bb7d2f6f15bc7', 'callType': 'call'}, 'traceAddress': [], 'result': {'output': '0x0000000000000000000000000000000000000000000000000000000000000001', 'gasUsed': '0x7310'}}], 'vmTrace': None}


In [22]:
import json

resp_string = str(resp.json())
resp_string = resp_string.replace("\'", "\"")
resp_string = resp_string.replace("None", "\"None\"")
resp_data = json.loads(resp_string)
print (resp_data)

{'result': [{'blockHash': '0x7eb25504e4c202cf3d62fd585d3e238f592c780cca82dacb2ed3cb5b38883add', 'traceAddress': [], 'blockNumber': 3068185, 'action': {'input': '0x', 'gas': '0x0', 'from': '0xaa7b131dc60b80d3cf5e59b5a21a666aa039c951', 'value': '0x4768d7effc3fbe', 'to': '0xd40aba8166a212d6892125f079c33e6f5ca19814', 'callType': 'call'}, 'transactionPosition': 0, 'subtraces': 0, 'type': 'call', 'result': {'output': '0x', 'gasUsed': '0x0'}, 'transactionHash': '0x07da28d752aba3b9dd7060005e554719c6205c8a3aea358599fc9b245c52f1f6'}, {'blockHash': '0x7eb25504e4c202cf3d62fd585d3e238f592c780cca82dacb2ed3cb5b38883add', 'traceAddress': [], 'blockNumber': 3068185, 'action': {'input': '0x', 'gas': '0x0', 'from': '0x4f11ba23bb526c0486d83c6a8f18f632f3fc172a', 'value': '0x446cde325fbfbe', 'to': '0x7ed1e469fcb3ee19c0366d829e291451be638e59', 'callType': 'call'}, 'transactionPosition': 1, 'subtraces': 0, 'type': 'call', 'result': {'output': '0x', 'gasUsed': '0x0'}, 'transactionHash': '0x056f11efb5da4ff7cf85